# notebook/02_data_transformation_to_silver

In [1]:
import os
import sys 
import logging

# Caminho para a raiz do seu projeto
project_root = os.path.abspath("..")  # Sobe um nível de pasta a partir de 'notebooks'

# Adiciona o caminho ao sys.path
if project_root not in sys.path:
    sys.path.insert(0, project_root)

from src.utils import functions as F
from config.configs import *
from config.spark_session import create_spark_session

## Spark Session

In [ ]:
spark = create_spark_session(HOST_ADDRESS, MINIO_ACCESS_KEY, MINIO_SECRET_KEY)

## Log configs

In [ ]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

logging.info("Starting convertions from Bronze(Delta) to Silver(Delta) - COVID19...")

## Path configs

In [4]:
input_prefix_layer_name = prefix_layer_name['1']  # bronze layer
input_path = lake_path['bronze']

output_prefix_layer_name = prefix_layer_name['2']  # silver layer
storage_output = lake_path['silver']
output_tabela_name = tabela_name['0']  # nivel agregação

output_full_name = f"{storage_output}{output_prefix_layer_name}{output_tabela_name}"

## Process

In [5]:
try:
    for table_name, query_input in tables_silver.items():
        table_name = F.convert_table_name(table_name)

        # Obter a consulta para o carregamento do arquivo
        query_input = F.get_query(table_name, input_path, input_prefix_layer_name, output_tabela_name, tables_silver)
        
        # Definir o caminho de saída
        storage_output = f'{output_full_name}'

        # Chamar a função process_table para processar e gravar os dados
        F.process_table(spark, query_input, output_full_name)
    
    logging.info("Process to silver completed!")
    
except Exception as e:
    logging.error(f'Error processing table: {str(e)}')

24/11/18 11:30:28 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
24/11/18 11:30:31 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
                                                                                2024-11-18 11:30:42,956 - INFO - Query 'SELECT
	location_key, country_name, date,
    population, new_confirmed, new_deceased, new_recovered,
    cumulative_confirmed, cumulative_deceased, cumulative_recovered,
    new_persons_vaccinated, cumulative_persons_vaccinated,
    new_hospitalized_patients, cumulative_hospitalized_patients,
    mobility_retail_and_recreation, aggregation_level
FROM
    delta.`s3a://bronze/covid19/bronze_covid19` 
WHERE 
    date IS NOT NULL AND 
    location_key IS NOT NULL
    
    ' successfully processed and saved to s3a://silver/covid19/silver_covid19
2024-11-1